You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Aggregate Operation:
o	Perform a simple "aggregate" operation on the retrieved launch data.
o	Aggregation Logic: Calculate the total number of launches for each year.
o	Return a print with the number of aggregate launches for each year
3.	Control Parameters and Debugging:
o	Include a variable at the beginning of the script to define the API endpoint URL, making it easily modifiable: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, starting aggregation, aggregation complete, number of years aggregated, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + aggregation). Print this time after the aggregation operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting aggregated summary dictionary (year counts) from the aggregate operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting aggregated JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
import requests
import logging
import time
import json

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define control parameters
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
HTTPBIN_POST_URL = "https://httpbin.org/post"

def retrieve_launch_data(api_url):
    """
    Retrieve launch data from the SpaceX API.
    Handles potential errors such as timeouts or non-200 status codes.
    """
    try:
        logging.info(f"Starting data retrieval from {api_url}")
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        logging.info(f"Data retrieval successful. Retrieved {len(data)} launches.")
        return data
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during data retrieval: {e}")
        return None

def aggregate_launch_data(launch_data):
    """
    Aggregate the launch data by year.
    Returns a dictionary with years as keys and the count of launches as values.
    """
    logging.info("Starting aggregation of launch data.")
    year_counts = {}
    for launch in launch_data:
        launch_year = launch['launch_year']
        if launch_year in year_counts:
            year_counts[launch_year] += 1
        else:
            year_counts[launch_year] = 1
    logging.info(f"Aggregation complete. Aggregated {len(year_counts)} years.")
    return year_counts

def upload_to_httpbin(data, post_url):
    """
    Serialize the aggregated data into JSON and upload it to httpbin.org.
    Logs the outcome of the upload operation.
    """
    try:
        logging.info(f"Starting upload to {post_url}")
        json_data = json.dumps(data)
        headers = {'Content-Type': 'application/json'}
        response = requests.post(post_url, data=json_data, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors
        logging.info(f"Upload successful. Status code: {response.status_code}")
        return True
    except requests.exceptions.RequestException as e:
        logging.error(f"Error during upload to httpbin: {e}")
        return False

# Main execution
if __name__ == "__main__":
    start_time_pipeline = time.time()

    # Step 1: Retrieve launch data
    launch_data = retrieve_launch_data(API_ENDPOINT_URL)
    if launch_data is None:
        logging.error("Failed to retrieve launch data. Exiting script.")
        exit(1)

    # Measure time for data retrieval + aggregation
    start_time_main_operations = time.time()

    # Step 2: Aggregate launch data
    year_counts = aggregate_launch_data(launch_data)
    for year, count in sorted(year_counts.items()):
        print(f"Year: {year}, Total Launches: {count}")

    # Measure and log execution time for main operations
    main_operations_time = time.time() - start_time_main_operations
    logging.info(f"Time taken for data retrieval and aggregation: {main_operations_time:.2f} seconds")

    # Step 3: Upload aggregated data to httpbin
    upload_successful = upload_to_httpbin(year_counts, HTTPBIN_POST_URL)

    # Measure and log total pipeline execution time
    total_pipeline_time = time.time() - start_time_pipeline
    logging.info(f"Total pipeline execution time: {total_pipeline_time:.2f} seconds")

    if not upload_successful:
        logging.error("Pipeline completed with errors.")